In [1]:
# Cell 1: Define waveform generation functions and display widgets

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Global variables to be used across cells
export_count = 1  # Export count for LUT naming
scaled_wave = None  # Store the waveform globally for export

# Function to generate core waveform
def generate_core_waveform(LUT_SIZE, freq, phase, ramp_steepness, waveform_type):
    t = (np.linspace(0, 1, LUT_SIZE, endpoint=False) * freq + phase) % 1
    if waveform_type == 'Sine':
        wave = np.sin(2 * np.pi * t)
    elif waveform_type == 'Ramp Down':
        wave = (1 - t) ** ramp_steepness
    elif waveform_type == 'Ramp Up':
        wave = t ** ramp_steepness
    elif waveform_type == 'Triangle':
        wave = 2 * np.abs(t - 0.5) - 1
    elif waveform_type == 'Square':
        wave = np.sign(np.sin(2 * np.pi * t))
    return wave

# Function to generate modulation waveform
def generate_mod_wave(LUT_SIZE, freq, phase, mod_type, duty_cycle):
    t = (np.linspace(0, 1, LUT_SIZE, endpoint=False) * freq + phase) % 1
    if mod_type == 'Sine':
        mod_wave = np.sin(2 * np.pi * t)
    elif mod_type == 'Ramp Down':
        mod_wave = 1 - t
    elif mod_type == 'Ramp Up':
        mod_wave = t
    elif mod_type == 'Triangle':
        mod_wave = 2 * np.abs(t - 0.5) - 1
    elif mod_type == 'Square':
        mod_wave = np.where(t < duty_cycle, 1, -1)  # Apply duty cycle
    return mod_wave

# Function to plot and save the waveform (also save as PNG)
def plot_lut(core_wave_type, mod_wave_type, core_freq, mod_freq, core_phase, mod_phase, ramp_steepness, mod_amplitude, mod_fadeIn, mod_fadeOut, duty_cycle, min_dac, max_dac, lut_size):
    global scaled_wave  # Declare global so it can be accessed in export functions
    LUT_SIZE = lut_size

    core_wave = generate_core_waveform(LUT_SIZE, core_freq, core_phase, ramp_steepness, core_wave_type)
    mod_wave = generate_mod_wave(LUT_SIZE, mod_freq, mod_phase, mod_wave_type, duty_cycle)

    fade_in_effect = np.linspace(0, 1, LUT_SIZE) ** mod_fadeIn
    fade_out_effect = np.linspace(1, 0, LUT_SIZE) ** mod_fadeOut
    combined_fade = np.minimum(fade_in_effect, fade_out_effect)

    combined_wave = core_wave + (mod_amplitude * combined_fade) * mod_wave
    normalized_wave = (combined_wave - np.min(combined_wave)) / (np.max(combined_wave) - np.min(combined_wave))
    scaled_wave = normalized_wave * (max_dac - min_dac) + min_dac

    # Plot the waveform
    plt.figure(figsize=(10, 4))
    plt.plot(scaled_wave)
    plt.title(f'Core: {core_wave_type}, Mod: {mod_wave_type}, Core Freq: {core_freq}, Mod Freq: {mod_freq}')
    plt.xlabel('Index')
    plt.ylabel('DAC Value')
    plt.grid(True)

    # Save the plot as a PNG file
    filename = f'myLUT_{export_count}.png'
    plt.savefig(filename)
    plt.show()
    print(f"Waveform plot saved as {filename}")

# Use interact to display the GUI controls and handle plotting
interact(plot_lut,
         core_wave_type=widgets.Dropdown(options=['Sine', 'Ramp Down', 'Ramp Up', 'Triangle', 'Square'], value='Ramp Down', description='core_wave_type'),
         core_freq=widgets.FloatSlider(min=1.0, max=8, step=0.25, value=1, description='core_freq'),
         core_phase=widgets.FloatSlider(min=0, max=1, step=0.01, value=0, description='core_phase'),
         ramp_steepness=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='core_ramp_steepness'),
         mod_wave_type=widgets.Dropdown(options=['Sine', 'Ramp Down', 'Ramp Up', 'Triangle', 'Square'], value='Sine', description='mod_wave_type'),
         mod_freq=widgets.FloatSlider(min=1.0, max=8, step=0.25, value=3.50, description='mod_freq'),
         mod_phase=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='mod_phase'),
         mod_amplitude=widgets.FloatSlider(min=0, max=1, step=0.01, value=1.0, description='mod_amplitude'),
         mod_fadeIn=widgets.FloatSlider(min=0.0, max=10, step=0.1, value=1.0, description='mod_fadeIn'),
         mod_fadeOut=widgets.FloatSlider(min=0.0, max=10, step=0.1, value=1.0, description='mod_fadeOut'),
         duty_cycle=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description='mod_duty_cycle_square'),
         min_dac=widgets.IntText(value=0, description='min_dac'),
         max_dac=widgets.IntText(value=3090, description='max_dac'),
         lut_size=widgets.IntText(value=4096, description='lut_size'))


interactive(children=(Dropdown(description='core_wave_type', index=1, options=('Sine', 'Ramp Down', 'Ramp Up',…

<function __main__.plot_lut(core_wave_type, mod_wave_type, core_freq, mod_freq, core_phase, mod_phase, ramp_steepness, mod_amplitude, mod_fadeIn, mod_fadeOut, duty_cycle, min_dac, max_dac, lut_size)>

In [2]:
# Cell 2: Export the LUT and waveform plot

def export_lut_for_arduino(lut):
    global export_count  # To maintain count across multiple exports

    # Truncate the float values to integers
    integer_lut = np.floor(lut).astype(int)

    # Construct the filename with incrementing count
    filename = f'myLUT_{export_count}.txt'

    # Arduino LUT declaration
    lut_name = f"myLUT_{export_count}"
    header = f"const uint16_t {lut_name}[LUT_SIZE] = {{\n"

    # Prepare the values, 10 per line
    lines = []
    for i in range(0, len(integer_lut), 10):
        line = ", ".join(map(str, integer_lut[i:i+10]))
        lines.append(f"  {line}")

    # Final content
    content = header + ",\n".join(lines) + "\n};\n"

    # Write to the file
    with open(filename, "w") as file:
        file.write(content)

    print(f"LUT exported to {filename}")

    # Increment the export count for future LUTs
    export_count += 1


In [12]:
# Cell 3: Export the current LUT (scaled_wave)
export_lut_for_arduino(scaled_wave)


LUT exported to myLUT_10.txt
